In [34]:
from pathlib import Path
from xml.dom.minidom import Document

import pandas as pd
import re

from exceptiongroup import catch
from sqlalchemy.engine import row
from sqlalchemy.testing.plugin.plugin_base import logging
from langchain_core.documents import Document


DATA_PATH = Path("/home/wby/projects/week5/data/recipes.csv")
BATCH_SIZE = 1000
KEY_FIELD = [
        "RecipeId",                  # 食谱唯一ID（关联片段与原数据，必选）
        "Name",                      # 食谱名称（生成回答时展示，必选）
        "RecipeIngredientQuantities",# 食材用量（如“500g”，核心检索字段）
        "RecipeIngredientParts",     # 食材名称（如“鸡肉”，核心检索字段）
        "RecipeCategory",            # 食物类别
        "RecipeInstructions",        # 烹饪步骤（生成回答的核心内容，必选）
        "TotalTime",                 # 总耗时（用户可能筛选“30分钟内”，可选）
        "Calories",                  # 热量（用户可能筛选“低热量”，可选）
        "RecipeServings"             # 份数（后续Agent单位转换用，可选）
    ]

# 食品时间对照表
null_cat_time_map = {
    # 第一类：食材/菜品类型
    "Papaya": 15,
    "Strawberry": 15,
    "Penne": 25,
    "Yam/Sweet Potato": 35,
    "Whole Turkey": 180,
    "Rabbit": 90,
    # 第二类：地域菜系
    "Japanese": 30,
    "Thai": 35,
    "Vietnamese": 30,
    "Lebanese": 45,
    "Swiss": 50,
    "Canadian": 45,
    # 第三类：饮食场景/需求
    "Kid Friendly": 20,
    "Potluck": 60,
    "Vegan": 35,
    "Native American": 40,
    "Medium Grain Rice": 30,
    "Elk": 90
}

# 食品数量对照表
serving_map = {
    # 第一类：食材量/菜品规模
    "Whole Turkey": 10,
    "Berries": 4,
    "Citrus": 4,
    "Strawberry": 4,
    "Papaya": 4,
    "Perch": 2,
    # 第二类：饮食场景/需求
    "Kid Friendly": 2,
    "Easy": 2,
    "No Cook": 2,
    "Lactose Free": 4,
    "Spicy": 4,
    # 第三类：地域/食材类型
    "Chinese": 4,
    "Hungarian": 4,
    "Southwest Asia (middle East)": 4,
    "Grains": 4
}

#食品卡路里对照表
calorie_map = {
    # 饮品/酱料类（低卡）
    "Beverages": 80,          # 拿铁咖啡约80大卡
    "Herbal Vinegar": 5,      # 草本醋几乎无热量，取5大卡
    "Gelatin": 70,            # 果冻约70大卡
    "Cookie Icing": 120,      # 糖霜约120大卡/份
    "Lemon Marmalade": 100,   # 柠檬酱约100大卡

    # 蔬菜/水果类（中低卡）
    "Vegetable": 60,          # 蔬菜派约60大卡
    "Peppers": 30,            # 烤甜椒约30大卡
    "Fruit": 50,              # 水果类约50大卡
    "Native American": 80,    # 南瓜约80大卡
    "Vegan": 90,              # 纯素南瓜泥约90大卡

    # 主食/肉类/甜点（中高卡）
    "Chicken Breast": 200,    # 炸鸡条约200大卡
    "Chicken": 220,           # 鸡肉料理约220大卡
    "Meat": 300,              # 烤肉约300大卡
    "Cheese": 180,            # 奶酪约180大卡
    "European": 250,          # 意大利面/烤面包约250大卡
    "Dessert": 350,           # 自制糕点约350大卡
    "Lunch/Snacks": 200,      # 三明治/芝士条约200大卡
    "Very Low Carbs": 150     # 低卡蜂蜜约150大卡（含少量糖）
}

In [35]:
# 用修复后的临时文件读取
batchs = pd.read_csv(
            DATA_PATH,
            chunksize=BATCH_SIZE,
            usecols=KEY_FIELD,
            encoding="utf-8",
            sep=",",  # 指定逗号分隔
            on_bad_lines="skip",  # 跳过极坏行
            engine="python"  # 必须用 python 引擎
        )

In [36]:

#初始化df的各种格式
def init_df(df):
    df['TotalTime'] =df["TotalTime"].apply(iso_to_minutes)
    df["Ingredient_list"] = df["RecipeIngredientParts"].apply(parse_r_list)
    df["Quantity_list"] = df["RecipeIngredientQuantities"].apply(parse_r_list)
    df['Ingredient_list'] = df['Ingredient_list'].apply(del_same)

#获取平均数
def getAvager(df,colum):
    avager = df.groupby('RecipeCategory')[colum].mean()
    return avager

# 时间转化为分钟--apply（已用于初始化）
def iso_to_minutes(iso_str):
    # 第一步：先处理“非字符串类型”（数字、布尔值等）
    if not isinstance(iso_str, str):
        # 如果是NaN，返回None（保持你原有的空值逻辑）
        if pd.isna(iso_str):
            return None
        # 如果是数字（如30、60.5），直接返回整数（假设数字已是分钟数）
        elif isinstance(iso_str, (int, float)):
            return round(iso_str) if iso_str >= 0 else None
        # 其他类型（如布尔值），返回None
        else:
            return None

    # 第二步：处理字符串类型（你的原有逻辑，保持不变）
    if iso_str.strip() == 'PT0S':  # 加strip()避免空字符干扰（如' PT0S '）
        return None

    pattern = r"(\d+\.?\d*)([DHMS])"
    matches = re.findall(pattern, iso_str)

    total_time = 0.0
    for time, unit in matches:
        try:  # 加try-except，避免time不是数字的情况（如'abcH'）
            num = float(time)
        except ValueError:
            continue  # 遇到无效数字，跳过该匹配

        if unit == 'D':
            total_time += num * 1440  # 天→分钟：1天=1440分钟
        elif unit == 'H':
            total_time += num * 60     # 小时→分钟
        elif unit == 'M':
            total_time += num          # 分钟直接加
        elif unit == 'S':
            total_time += num / 60     # 秒→分钟

    # 避免出现0分钟（可能是匹配失败），返回None
    return round(total_time) if total_time > 0 else None

# 清洗TotalTime数据
def rush_time(df):

    #先进行人工表的输入
    for cat,time in null_cat_time_map.items():
        mask = (df['RecipeCategory'] == cat) & (df['TotalTime'].isna())
        df.loc[mask, 'TotalTime'] = time

    #后进行平均值的摄入
    avager = getAvager(df, 'TotalTime')
    for cat,time in avager.items():
        if pd.isna(time):
            continue
        mask = (df['RecipeCategory'] == cat) & (df['TotalTime'].isna())
        df.loc[mask, 'TotalTime'] = time

    #最后进行全局中位数填充
    global_median_time = df['TotalTime'].median()
    mask = df['TotalTime'].isna()
    df.loc[mask, 'TotalTime'] = global_median_time

# 清洗RecipeServings数据
def rush_serving(df):
    #先考虑人工表
    for cat,num in serving_map.items():
        mask = (df['RecipeCategory'] == cat) & (df['RecipeServings'].isna())
        df.loc[mask, 'RecipeServings'] = num

    #再考虑平均值
    avager = getAvager(df, 'RecipeServings').round()
    for cat,num in avager.items():
        if pd.isna(num):
            continue
        mask = (df['RecipeCategory'] == cat) & (df['RecipeServings'].isna())
        df.loc[mask, 'RecipeServings'] = num

    #最后用全局中位数
    global_median_num = df['RecipeServings'].median()
    mask = df['RecipeServings'].isna()
    df.loc[mask, 'RecipeServings'] = global_median_num

# 清洗卡路里数据
def rush_calorie(df):
    df['Calories'] = pd.to_numeric(df['Calories'], errors='coerce')
    for cat,cal in calorie_map.items():
        mask = (df['RecipeCategory'] == cat) & ((df['Calories']<=0)|(df['Calories'].isna()))
        df.loc[mask, 'Calories'] = cal

    avager = getAvager(df, 'Calories').round()
    for cat,num in avager.items():
        if pd.isna(num) or num <= 0:
            continue
        mask = (df['RecipeCategory'] == cat) & ((df['Calories']<=0)|(df['Calories'].isna()))
        df.loc[mask, 'Calories'] = num

    global_median_cal = df['Calories'].median()
    mask = df['Calories'].isna()|df['Calories']<=0
    df.loc[mask, 'Calories'] = global_median_cal

#分割数组的函数--apply(已用于初始化)
def parse_r_list(r_str):
    if pd.isna(r_str):
        return []

    cleaned = re.sub(r'^c\(|(?<=\D)\)$', '', r_str)  # 去掉 "c(" 和 非数字结尾的 ")"
    raw_items = cleaned.split(',') #['orange','"apple"','nan']

    valid_items = []
    for item in raw_items:
        strs = item.strip().strip('"')
        if strs != '' and strs.upper()!='NA':
            valid_items.append(strs)
    return valid_items

# 食谱去重--apply(已用于初始化)
def del_same(row):
    """
    pd.isna()解决的是元素是否缺失，而不是判定数组是否为空
    # 对列表中的每个元素判断是否为缺失值
    result = pd.isna(['a', 'b', 'na'])
    print(result)  # 输出：[False False False]
    row = []
    print(pd.isna(row))  # 输出：array([], dtype=bool)
    """
    if len(row) == 0 or not row:
        return []

    maps = dict()
    for v in row:
        if v not in maps:
            maps[v] = v
    return list(maps.values())

# 多余者切除--已用于切除函数里
def cut_long(longer_list,smaller_list):
    return longer_list[:len(smaller_list)]

# 切除函数
def cut_list(df):
    #空表防护
    df['Ingredient_list'] = df['Ingredient_list'].apply(lambda x: ['无'] if len(x) == 0 else x)
    df['Quantity_list'] = df['Quantity_list'].apply(lambda x: ['适量'] if len(x) == 0 else x)
    #先切割食谱大于食量的
    df['Ingredient_len'] = df['Ingredient_list'].apply(len)
    df['Quantity_len'] = df['Quantity_list'].apply(len)
    mask = (df['Ingredient_len']>df['Quantity_len'])
    df.loc[mask,'Ingredient_list'] = df.apply(lambda row: cut_long(
        row['Ingredient_list'],
        row['Quantity_list']
    ), axis=1)
    mask = (df['Ingredient_len']<df['Quantity_len'])
    df.loc[mask,'Quantity_list'] = df.apply(lambda row: cut_long(
        row['Quantity_list'],
        row['Ingredient_list']
    ),axis=1)

    #更新
    df['Ingredient_len'] = df['Ingredient_list'].apply(len)
    df['Quantity_len'] = df['Quantity_list'].apply(len)

#跳过错误行
def skip_and_warn(bad_line):
    print(f"跳过错误行：{bad_line}")  # 打印错误行内容
    return None  # 返回None表示跳过该行

#转为文本结构---apply
def conver_row_to_text(row):
    basic_info=f"""
RecipeId:{row['RecipeId']}
Name:{row['Name']}
TotalTime:{row['TotalTime']}
RecipeCategory:{row['RecipeCategory']}
Calories:{row['Calories']}
RecipeServings:{row['RecipeServings']}
""".strip()

    ingredient_parirs = []
    for ing,quant in zip(row['Ingredient_list'],row['Quantity_list']):
        strs = f"{ing}:{quant}"
        ingredient_parirs.append(strs)
    ingredient_info = f"Ingredients and Quantities:"+'\n' + '\n'.join(ingredient_parirs)

    instructions = row['RecipeInstructions'] if pd.notna(row['RecipeInstructions']) else 'No detailed steps available'
    instruction_info = instructions.strip().lstrip('c(').rstrip(')')
    steps_strings = instruction_info.split('", "')
    steps = []

    for s in steps_strings:
        clean_step = s.strip().strip('"')
        steps.append(clean_step)

    instruction_complete_list = [f'{i+1}. {step}' for i,step in enumerate(steps)]
    instruction_comlete = 'Cooking steps:' + '\n' + '\n'.join(instruction_complete_list)

    full_text = basic_info + '\n' + ingredient_info.strip() + '\n' + instruction_comlete.strip()
    return full_text

#建立元数据 --apply
def create_baseInfor(row):
    return {
        'RecipeId':row['RecipeId'],
        'Name':row['Name'],
        'TotalTime':row['TotalTime'],
        'RecipeCategory':row['RecipeCategory'],
        'Calories':row['Calories'],
        'RecipeServings':row['RecipeServings']
    }


In [ ]:
#若要测试50万数据，则调用这个
try:
    nums = 0
    for batch in batchs:
        try:
            print(f"开始{nums+1}批数据")
            init_df(batch)
            rush_time(batch)
            rush_calorie(batch)
            rush_serving(batch)
            cut_list(batch)
            batch.dropna(inplace=True)#清洗后还有的数据不要了
            print(f"第{nums+1}批数据情况如下:"
                  f"食谱不等者{batch[batch['Ingredient_len'] != batch['Quantity_len']].shape[0]}  "
                  f"卡路里为0者{batch[batch['Calories'] == 0].shape[0]}  "
                  f"TotalTime为0者{batch['TotalTime'].isna().sum()}  "
                  f"Recipes为na者{batch['RecipeServings'].isna().sum()}")

        except Exception as e:
            print(f"第{nums+1}批出错：{e}")
        finally:
            del batch
            import gc; gc.collect()

        #直接上强度
        nums += 1
        """if nums == 8:
            break"""
except Exception as e:
    print(f"读取阶段出错：{e}")

开始1批数据
第1批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始2批数据
第2批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始3批数据
第3批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始4批数据
第4批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始5批数据
第5批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始6批数据
第6批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始7批数据
第7批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0


In [37]:
#若要测试，则用这个代码
nums = 0
datalist = []
for batch in batchs:
    print(f"开始{nums+1}批数据")
    init_df(batch)
    rush_time(batch)
    rush_calorie(batch)
    rush_serving(batch)
    cut_list(batch)
    batch.dropna(inplace=True)#清洗后还有的数据不要了
    print(f"第{nums+1}批数据情况如下:"
          f"食谱不等者{batch[batch['Ingredient_len'] != batch['Quantity_len']].shape[0]}  "
           f"卡路里为0者{batch[batch['Calories'] == 0].shape[0]}  "
          f"TotalTime为0者{batch['TotalTime'].isna().sum()}  "
          f"Recipes为na者{batch['RecipeServings'].isna().sum()}")
    datalist.append(batch)
    #直接上强度
    nums += 1
    if nums == 8:
        break

df = pd.concat(datalist, ignore_index=True)

开始1批数据
第1批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始2批数据
第2批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始3批数据
第3批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始4批数据
第4批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始5批数据
第5批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始6批数据
第6批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始7批数据
第7批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0
开始8批数据
第8批数据情况如下:食谱不等者0  卡路里为0者0  TotalTime为0者0  Recipes为na者0


In [5]:

df['text'] = df.apply(conver_row_to_text, axis=1)
df['baseInfo'] = df.apply(create_baseInfor, axis=1)
for i in range(3):
    print(f"第{i+1}批数据\n{df.iloc[i]['text']}")
    print(f"对应元数据：{df.iloc[i]['baseInfo']}")


第1批数据
RecipeId:38
Name:Low-Fat Berry Blue Frozen Dessert
TotalTime:1485.0
RecipeCategory:Frozen Desserts
Calories:170.9
RecipeServings:4.0
Ingredients and Quantities:
blueberries:4
granulated sugar:1/4
vanilla yogurt:1
lemon juice:1
Cooking steps:
1. Toss 2 cups berries with sugar.
2. Let stand for 45 minutes, stirring occasionally.
3. Transfer berry-sugar mixture to food processor.
4. Add yogurt and process until smooth.
5. Strain through fine sieve. Pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). Freeze uncovered until edges are solid but centre is soft.  Transfer to processor and blend until smooth again.
6. Return to pan and freeze until edges are solid.
7. Transfer to processor and blend until smooth again.", 
"Fold in remaining 2 cups of blueberries.
8. Pour into plastic mold and freeze overnight. Let soften slightly to serve.
对应元数据：{'RecipeId': 38, 'Name': 'Low-Fat Berry Blue Frozen Dessert', 'TotalTime': 1485.0, 'RecipeC

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


test_df = df.iloc[:10]
sample_df = df.iloc[:1]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3000,
    chunk_overlap = 50,
    separators=['\n\n','\n',' ',', ']
)


chunks = Document(
    page_content = sample_df['text'].iloc[0],
    metadata = sample_df['baseInfo'].iloc[0]
)

print(chunks)

page_content='RecipeId:38
Name:Low-Fat Berry Blue Frozen Dessert
TotalTime:1485.0
RecipeCategory:Frozen Desserts
Calories:170.9
RecipeServings:4.0
Ingredients and Quantities:
blueberries:4
granulated sugar:1/4
vanilla yogurt:1
lemon juice:1
Cooking steps:
1. Toss 2 cups berries with sugar.
2. Let stand for 45 minutes, stirring occasionally.
3. Transfer berry-sugar mixture to food processor.
4. Add yogurt and process until smooth.
5. Strain through fine sieve. Pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). Freeze uncovered until edges are solid but centre is soft.  Transfer to processor and blend until smooth again.
6. Return to pan and freeze until edges are solid.
7. Transfer to processor and blend until smooth again.", 
"Fold in remaining 2 cups of blueberries.
8. Pour into plastic mold and freeze overnight. Let soften slightly to serve.' metadata={'RecipeId': 38, 'Name': 'Low-Fat Berry Blue Frozen Dessert', 'TotalTime': 1485

In [6]:
def create_chunks(df):
    chunks_list = []
    for i in range(df.shape[0]):
        chunks = Document(
            page_content = df['text'].iloc[i],
            metadata = df['baseInfo'].iloc[i],
        )
        chunks_list.append(chunks)
    return chunks_list

chunks_list = create_chunks(df)

for i in range(6):
    print(chunks_list[i])
    print('\n')

page_content='RecipeId:38
Name:Low-Fat Berry Blue Frozen Dessert
TotalTime:1485.0
RecipeCategory:Frozen Desserts
Calories:170.9
RecipeServings:4.0
Ingredients and Quantities:
blueberries:4
granulated sugar:1/4
vanilla yogurt:1
lemon juice:1
Cooking steps:
1. Toss 2 cups berries with sugar.
2. Let stand for 45 minutes, stirring occasionally.
3. Transfer berry-sugar mixture to food processor.
4. Add yogurt and process until smooth.
5. Strain through fine sieve. Pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). Freeze uncovered until edges are solid but centre is soft.  Transfer to processor and blend until smooth again.
6. Return to pan and freeze until edges are solid.
7. Transfer to processor and blend until smooth again.", 
"Fold in remaining 2 cups of blueberries.
8. Pour into plastic mold and freeze overnight. Let soften slightly to serve.' metadata={'RecipeId': 38, 'Name': 'Low-Fat Berry Blue Frozen Dessert', 'TotalTime': 1485

In [38]:
import os
import torch
from transformers import AutoModel, AutoTokenizer


# 1. 本地模型路径（你的实际路径）
MODEL_PATH = "/home/wby/projects/model/all-MiniLM-L6-v2"

# 2. 检查GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModel.from_pretrained(MODEL_PATH, local_files_only=True).to(device)

def get_embedding(text, tokenizer, model, device):
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)
    # 前向传播
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)
    # 取[CLS] token的向量（all-MiniLM默认用这个做句子嵌入）
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    # 向量归一化（和之前的逻辑一致）
    normalized_embedding = torch.nn.functional.normalize(cls_embedding, p=2, dim=1)
    # 转成numpy数组返回
    return normalized_embedding.cpu().numpy()[0].tolist()


In [39]:
def batch_generate_embedding(chunks_list, tokenizer, model, device):
    embeddings_list = []
    doc_ids_list = []
    metadatas_list = []
    print(f"开始批量处理，共{len(chunks_list)}个Document")
    for i,doc in enumerate(chunks_list):
        text = doc.page_content
        embedding = get_embedding(text, tokenizer, model, device)
        recipe_id = doc.metadata.get('RecipeId',f"unknown_{i}")
        doc_id = f"{recipe_id}_{i}"
        metadata = doc.metadata

        embeddings_list.append(embedding)
        doc_ids_list.append(doc_id)
        metadatas_list.append(metadata)

        if (i + 1) % 500 == 0:
            print(f"已处理 {i+ 1}/{len(chunks_list)} 个Document")

    print(f"批量处理完成！生成{len(embeddings_list)}个384维向量")
    return embeddings_list, doc_ids_list, metadatas_list



In [40]:
test = batch_generate_embedding(chunks_list, tokenizer, model, device)

NameError: name 'chunks_list' is not defined

In [41]:
#该脚本是用来创建chroma数据库的
import logging
from pathlib import Path
from chromadb import PersistentClient, Documents
from chromadb.config import Settings

LOG_DIR = '/home/wby/projects/week5/log'
LOG_FILE = 'chroma_store_log.txt'
LOG_FULL_PATH = Path(LOG_DIR) / LOG_FILE

logging.basicConfig(
    filename='/home/wby/projects/week5/log/chroma_store_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

def init_chroma(
        db_path: str = '/home/wby/projects/week5/chroma_db/chroma_recipe_db',
        collection_name: str = 'recipe_50w_384d_ver2'
)-> "PersistentClient.Collection":
    """
    初始化chroma数据库，不存在则创建，存在则直接获取
    :param db_path: chroma数据库的持久化路径
    :param collection_name: chroma数据库的集合名称
    :return: 返回chroma对象
    """
    # 创建持久化客户端
    chroma_client = PersistentClient(
        db_path,
        settings=Settings(
            anonymized_telemetry=False,  # 关闭匿名统计
            allow_reset=False,  # 禁止重置数据库（防误删）
            persist_directory=db_path  # 明确持久化路径
        )
    )

    #collection也有元数据
    collection = chroma_client.get_or_create_collection(
        collection_name,
        metadata={
            'description': '50万食谱数据',
            'vector_dimension': 384,
            "model_name": "nreimers/MiniLM-L6-H384-uncased",  # 记录用的模型
            "data_source": "recipes.csv"   # 数据来源
        }
    )

    # 打印/记录初始化信息
    logging.info(f"✅ Chroma初始化完成！")
    logging.info(f"  - 存储路径：{db_path}")
    logging.info(f"  - 集合名称：{collection_name}")
    logging.info(f"  - 当前集合已有向量数：{collection.count()}")  # 查看是否有历史数据
    print(f"Chroma初始化完成！当前集合已有向量数：{collection.count()}")
    return collection

recipe_collection = init_chroma()

Chroma初始化完成！当前集合已有向量数：521763


In [42]:
print(recipe_collection)

name='recipe_50w_384d_ver2' id=UUID('ab749f10-f397-4c99-8874-6f9d2acd9c61') metadata={'data_source': 'recipes.csv', 'description': '50万食谱数据', 'model_name': 'nreimers/MiniLM-L6-H384-uncased', 'vector_dimension': 384} tenant='default_tenant' database='default_database'


In [13]:
def batch_store_to_chroma(
    collection,  # 第一步初始化的Chroma集合对象
    chunks_list: list,  # 你的Document列表（create_chunks生成的）
    tokenizer, model, device,
    batch_size: int = 2000  # 每批存入数量（内存够可改2000）
):
    """
    批量将Embedding+ID+元数据存入Chroma，支持断点续跑
    :param collection: Chroma集合对象
    :param chunks_list: Document列表
    :param tokenizer/model/device: 初始化好的模型组件（复用，避免重复加载）
    :param batch_size: 每批处理数量
    """
    total_docs = len(chunks_list)
    total_batches = (total_docs // batch_size) + 1  # 总批数
    logging.info(f"开始存入Chroma：共{total_docs}个Document，分{total_batches}批处理")
    print(f"开始存入Chroma：共{total_docs}个Document，分{total_batches}批处理")

    # 遍历每批数据
    for batch_idx in range(total_batches):
        # 1. 计算当前批的起止索引
        start = batch_idx * batch_size
        end = min((batch_idx + 1) * batch_size, total_docs)  # 最后一批避免越界
        current_batch = chunks_list[start:end]
        current_batch_size = len(current_batch)

        if current_batch_size == 0:
            logging.info(f"⚠️  第{batch_idx+1}批无数据，跳过")
            continue

        try:
            # 2. 调用你的函数生成当前批的Embedding+ID+元数据
            embeddings, doc_ids, metadatas = batch_generate_embedding(
                chunks_list=current_batch,
                tokenizer=tokenizer,
                model=model,
                device=device
            )

            doc_texts = [doc.page_content for doc in current_batch]  # 关键：提取每个Document的文本
            # 3. 存入Chroma（核心API）
            collection.add(
                embeddings=embeddings,  # 384维向量列表
                ids=doc_ids,            # 唯一ID列表（RecipeId_索引，不重复）
                metadatas=metadatas,    # 元数据列表（食谱信息，用于筛选）
                documents=doc_texts     # 新增：存入文本，检索时才能返回
            )

            # 5. 记录进度
            processed_docs = end  # 已处理的总数量
            progress = (processed_docs / total_docs) * 100  # 进度百分比
            logging.info(f"✅ 第{batch_idx+1}/{total_batches}批存入成功！")
            logging.info(f"  - 处理范围：{start+1}-{end}/{total_docs}")
            logging.info(f"  - 当前进度：{round(progress, 2)}%")
            logging.info(f"  - 集合累计向量数：{collection.count()}")
            print(f"第{batch_idx+1}/{total_batches}批存入成功！进度：{round(progress, 2)}%，累计向量数：{collection.count()}")

        except Exception as e:
            # 异常捕获：某批出错不中断，记录错误后继续下一批
            error_msg = f"❌ 第{batch_idx+1}批存入失败！范围：{start+1}-{end}，错误：{str(e)}"
            logging.error(error_msg)
            print(error_msg)
            # 打印详细错误栈（方便定位问题，如ID重复、向量维度错）
            import traceback
            traceback.print_exc()
            continue

    # 全部完成后记录最终状态
    final_count = collection.count()
    logging.info(f"🎉 所有批次处理完成！")
    logging.info(f"  - 总输入Document数：{total_docs}")
    logging.info(f"  - Chroma最终向量数：{final_count}")
    logging.info(f"  - 存入成功率：{round((final_count / total_docs) * 100, 2)}%")
    print(f"\n🎉 所有批次处理完成！")
    print(f"  - 总输入Document数：{total_docs}")
    print(f"  - Chroma最终向量数：{final_count}")
    print(f"  - 存入成功率：{round((final_count / total_docs) * 100, 2)}%")

batch_store_to_chroma(recipe_collection,chunks_list, tokenizer, model, device)

开始存入Chroma：共7997个Document，分4批处理
开始批量处理，共2000个Document
已处理 500/2000 个Document
已处理 1000/2000 个Document
已处理 1500/2000 个Document
已处理 2000/2000 个Document
批量处理完成！生成2000个384维向量
第1/4批存入成功！进度：25.01%，累计向量数：2000
开始批量处理，共2000个Document
已处理 500/2000 个Document
已处理 1000/2000 个Document
已处理 1500/2000 个Document
已处理 2000/2000 个Document
批量处理完成！生成2000个384维向量
第2/4批存入成功！进度：50.02%，累计向量数：4000
开始批量处理，共2000个Document
已处理 500/2000 个Document
已处理 1000/2000 个Document
已处理 1500/2000 个Document
已处理 2000/2000 个Document
批量处理完成！生成2000个384维向量
第3/4批存入成功！进度：75.03%，累计向量数：6000
开始批量处理，共1997个Document
已处理 500/1997 个Document
已处理 1000/1997 个Document
已处理 1500/1997 个Document
批量处理完成！生成1997个384维向量
第4/4批存入成功！进度：100.0%，累计向量数：7997

🎉 所有批次处理完成！
  - 总输入Document数：7997
  - Chroma最终向量数：7997
  - 存入成功率：100.0%


In [44]:
def test_chroma_retrieval(collection, tokenizer, model, device):
    test_query = "ice cream"
    query_emb = get_embedding(test_query, tokenizer, model, device)

    # 关键：添加where参数，指定筛选条件（只保留TotalTime < 30的食谱）
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=4,
        include=["metadatas", "documents", "distances"],
        where={"TotalTime": {"$lt": 30}}  # $lt表示“小于”，对应查询中的“< 30”
    )

    print("\n=== 检索测试结果 ===")
    for i, (doc_id, distance, meta, doc_text) in enumerate(zip(
        results["ids"][0], results["distances"][0],
        results["metadatas"][0], results["documents"][0]
    )):
        print(f"\n第{i+1}个相似结果：")
        print(f"ID：{doc_id}")
        print(f"相似度：{round(distance, 4)}")
        print(f"食谱名称：{meta['Name']} | 耗时：{meta['TotalTime']}分钟")  # 打印耗时，验证是否符合条件
        print(f"文本内容：{doc_text}")

# 调用测试函数
test_chroma_retrieval(recipe_collection, tokenizer, model, device)


=== 检索测试结果 ===

第1个相似结果：
ID：59719_1479
相似度：0.1652
食谱名称：Homemade Ice Cream - No-Cook Version | 耗时：10.0分钟
文本内容：RecipeId:59719
Name:Homemade Ice Cream - No-Cook Version
TotalTime:10.0
RecipeCategory:Frozen Desserts
Calories:466.7
RecipeServings:12.0
Ingredients and Quantities:
granulated sugar:2 1/2
eggs:6
sweetened condensed milk:1
evaporated milk:2
vanilla extract:1
milk:1 1/2 - 2
sugar:1 -3
Cooking steps:
1. Combine first 5 ingredients and 1 quart of the milk, adding fruit if desired (optional).
2. Pour into a 2-gallon ice cream freezer.
3. Add enough of the remaining 1-quart milk to fill the ice cream canister to the fill line on the canister.
4. You may not need all of this last quart.
5. Do not over-fill the canister.
6. Make ice cream according to manufacturer's instructions for the ice cream freezer/churn.
7. VARIATION: For Peppermint and Chocolate: Crush 1 small bag of peppermint candy pieces and add 1 small (6 oz) bag of chocolate mini-chips.", 
"Add marshmallows and/or nuts, i

In [23]:
import chromadb

chroma_client = PersistentClient(
    path='/home/wby/projects/week5/chroma_db/chroma_recipe_db',
    settings=chromadb.config.Settings(
        anonymized_telemetry=False,
        allow_reset=False
    )
)

In [24]:
collections = chroma_client.list_collections()
print(collections)

[Collection(name=recipe_50w_384d_ver2)]


In [31]:
strs = """
[
            {
               "Name": "Best Apple Pie Ever",
               "Calories": 396.4,
               "Total Time": 95.0,
               "Reason": "This recipe is perfect for those who want a classic apple pie with a buttery crust. It takes 95 minutes to prepare and serves 8 people. The ingredients include flour, salt, butter, margarine, cinnamon, and cold water.",
               "taste": "sweet"
            },
            {
               "Name": "The Best Apple Pie",
               "Calories": 568.0,
               "Total Time": 85.0,
               "Reason": "This recipe is an excellent choice for those who love a rich and flavorful apple pie. It takes 85 minutes to prepare and serves 6 people. The ingredients include apples, butter, sugar, flour, cinnamon, and ground nutmeg.",
               "taste": "sweet"
            },
            {
               "Name": "Worlds Best Apple Pie",
               "Calories": 243.1,
               "Total Time": 80.0,
               "Reason": "If you're looking for a quick and easy apple pie that still tastes delicious, this recipe is perfect for you. It takes only 80 minutes to prepare and serves 8 people. The ingredients include apples, lemon juice, white sugar, flour, salt, and nutmeg.",
               "taste": "sweet"
            }
         ]
```
"""

In [27]:
import json
import re
def parse_model_output(raw_output):
    try:
        # 第一步：先尝试直接解析原始输出（正常JSON直接返回）
        return json.loads(raw_output.strip())
    except json.JSONDecodeError:
        cleaned_output = raw_output.strip()
        # 1. 去掉开头/结尾的 ``` 标记
        cleaned_output = re.sub(r'^```(json)?\s*', '', cleaned_output)
        cleaned_output = re.sub(r'\s*```$', '', cleaned_output)
        # 2. 关键修复：删除末尾多余的 ] 或 }（解决双重闭合问题）
        cleaned_output = cleaned_output.rstrip('}]').rstrip(',').strip() + (']' if cleaned_output.lstrip().startswith('[') else '}')
        # 3. 去掉数组/对象内的多余逗号
        cleaned_output = re.sub(r',\s*([}\]])', r'\1', cleaned_output)
        # 4. 清理多余空白（不破坏结构）
        cleaned_output = re.sub(r'\n+|\t+', ' ', cleaned_output)
        # 再次尝试解析
        try:
            return json.loads(cleaned_output)
        except json.JSONDecodeError as e:
            print(f"解析失败：{e}，清洗后内容：{cleaned_output}")
            return [
                {
                    "Name": "Data Parse Error",
                    "Calories": 0,
                    "Total Time": 0,
                    "Reason": "Failed to parse recipe data. Please try again.",
                    "taste": "unknown"
                }
            ]

In [28]:
try:
    final_s = json.loads(strs)

except:
    final_s = parse_model_json(strs)
print(final_s)

[{'Name': 'Best Apple Pie Ever', 'Calories': 396.4, 'Total Time': 95.0, 'Reason': 'This recipe is perfect for those who want a classic apple pie with a buttery crust. It takes 95 minutes to prepare and serves 8 people. The ingredients include flour, salt, butter, margarine, cinnamon, and cold water.', 'taste': 'sweet'}, {'Name': 'The Best Apple Pie', 'Calories': 568.0, 'Total Time': 85.0, 'Reason': 'This recipe is an excellent choice for those who love a rich and flavorful apple pie. It takes 85 minutes to prepare and serves 6 people. The ingredients include apples, butter, sugar, flour, cinnamon, and ground nutmeg.', 'taste': 'sweet'}, {'Name': 'Worlds Best Apple Pie', 'Calories': 243.1, 'Total Time': 80.0, 'Reason': "If you're looking for a quick and easy apple pie that still tastes delicious, this recipe is perfect for you. It takes only 80 minutes to prepare and serves 8 people. The ingredients include apples, lemon juice, white sugar, flour, salt, and nutmeg.", 'taste': 'sweet'}]


In [32]:
strs2 = """
[
             {
                 "Name": "Lentil Praline Ice Cream",
                 "Calories": "320.2 卡",
                 "Total Time": "3060.0 分钟",
                 "Reason": "This recipe uses lentils as a base and pralines for added sweetness. The total time is 3 hours, which may be too long for some users. However, if you want a healthier option with less sugar, this recipe could be a good choice.",
                 "taste": "sweet"
             },
             {
                 "Name": "Chic Chef's Citrus Sorbetto",
                 "Calories": "612.9 卡",
                 "Total Time": "10.0 分钟",
                 "Reason": "This recipe uses fresh citrus fruits and requires no cooking. The total time is very short, making it a great option for busy individuals. It also has relatively low calories, making it a healthy choice.",
                 "taste": "sour"
             }
]
"""

In [33]:
try:
    final_s = json.loads(strs2)

except:
    final_s = parse_model_json(strs2)
print(final_s)

[{'Name': 'Lentil Praline Ice Cream', 'Calories': '320.2 卡', 'Total Time': '3060.0 分钟', 'Reason': 'This recipe uses lentils as a base and pralines for added sweetness. The total time is 3 hours, which may be too long for some users. However, if you want a healthier option with less sugar, this recipe could be a good choice.', 'taste': 'sweet'}, {'Name': "Chic Chef's Citrus Sorbetto", 'Calories': '612.9 卡', 'Total Time': '10.0 分钟', 'Reason': 'This recipe uses fresh citrus fruits and requires no cooking. The total time is very short, making it a great option for busy individuals. It also has relatively low calories, making it a healthy choice.', 'taste': 'sour'}]
